# คู่มือการตั้งค่าและใช้งาน YOLOv7 สำหรับ Ragnarok Object Detection

## 1. ดาวน์โหลด YOLOv7
ดาวน์โหลดไลบรารี YOLOv7 จาก GitHub:
- [YOLOv7 GitHub Repository](https://github.com/WongKinYiu/yolov7.git)

## 2. สร้างและตั้งค่าสภาพแวดล้อม (Environment)
1. สร้าง Environment ใหม่ด้วย Conda:
   ```bash
   conda create -n yolov7 python=3.9
   ```
2. เปิดใช้งาน Environment:
   ```bash
   conda activate yolov7
   ```
3. ติดตั้งไลบรารีจาก `requirements.txt`:
   ```bash
   pip install -r requirements.txt
   ```
4. ติดตั้ง PyTorch:
   ```bash
   conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
   ```

## 3. การทดสอบ YOLOv7 เบื้องต้น
1. ทดสอบการทำงานของ YOLOv7:
   ```bash
   python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images
   ```
2. ตรวจสอบว่า GPU ใช้งานได้:
   ```bash
   python detect.py --device 0
   ```

## 4. เตรียมข้อมูลสำหรับเทรนโมเดล
### การจัดการข้อมูล
1. ใช้ `screenshot/read_capture_video.py` เพื่อสร้างภาพจากวิดีโอ.
2. หากต้องการทำ Object Detection:
   - ใช้ [MakeSense](https://www.makesense.ai/) เพื่อ Annotate รูปภาพ และ Export เป็น YOLO Format.

### โครงสร้างโฟลเดอร์
สร้างโฟลเดอร์ `train_data` และจัดเรียงข้อมูลดังนี้:
```
train_data/
    images/
        train/
        val/
    labels/
        train/
        val/
```
- แบ่งภาพและไฟล์ Annotation เป็น 80% สำหรับ Train และ 20% สำหรับ Validation.

## 5. สร้างไฟล์ Data Config
1. คัดลอกไฟล์ `coco.yaml` และตั้งชื่อใหม่ เช่น `ragnarok.yaml`.
2. อัปเดตข้อมูลในไฟล์:
   - แก้ชื่อ Dataset และระบุที่ตั้งไฟล์.

## 6. เทรนโมเดล
### เทรนโมเดลขนาดใหญ่
```bash
set KMP_DUPLICATE_LIB_OK=TRUE
python train.py --workers 10 --device 0 --batch-size 5 --data data/ragnarok.yaml --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --name Ragnarok --hyp data/hyp.scratch.p5.yaml --epochs 100
```

### เทรนโมเดลขนาดเล็ก (Tiny)
```bash
set KMP_DUPLICATE_LIB_OK=TRUE
python train.py --workers 8 --device 0 --batch-size 8 --data data/ragnarok.yaml --cfg cfg/training/yolov7-tiny.yaml --weights 'weights/yolov7-tiny.pt' --name Ragnarok_Tiny --hyp data/hyp.scratch.tiny.yaml --epochs 50
```

## 7. การปรับค่า `--workers`
- ค่าที่เหมาะสมคือ 50-75% ของจำนวน Threads บน CPU.
- ตัวอย่าง:
  - CPU 12 Threads: `--workers = 6-8`

## 8. ทดสอบโมเดลที่เทรน
### โมเดลขนาดใหญ่
```bash
python detect.py --weights runs/train/Ragnarok/weights/best.pt --conf 0.25 --source ../testdata/test.mp4
```

### โมเดลขนาดเล็ก (Tiny)
```bash
python detect.py --weights runs/train/Ragnarok_Tiny/weights/ragnarok_tiny.pt --conf 0.25 --source ../testdata/test.mp4
```

## หมายเหตุ
- หากเกิดปัญหาเกี่ยวกับการจับหน้าจอ ให้รันโปรแกรมในโหมด Admin.
- ปรับค่า `--conf` หรือพารามิเตอร์อื่นเพื่อเพิ่มความแม่นยำตามความต้องการ.